In [52]:
import networkx as nx
import matplotlib.pyplot as plt



def read_graph(fake = True):
    global edges
    if(fake):
        edges = [(1, 2), (1, 3), (2, 3), (2, 4), (3, 4)]
        G = nx.Graph()
        G.add_edges_from(edges)
        return G
    else:
        file_path = 'data/example1.dat'

        # Reading the file and creating edges
        with open(file_path, 'r') as file:
            edges = [tuple(map(int, line.strip().split(','))) for line in file if line.strip()]

        G = nx.Graph()
        G.add_edges_from(edges)

        return G


In [53]:
G =read_graph(False)

In [54]:
G

In [55]:

# Get adjacency matrix
A = nx.adjacency_matrix(G).toarray()
A

array([[0, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [56]:
import numpy as np
from numpy.linalg import eig


w,v=eig(A)
print('E-value:', w)
print('E-vector', v)

E-value: [ 1.00604828e+01  7.30763087e+00  6.00367943e+00  5.55345917e+00
  5.29428632e+00  4.96163327e+00  4.76850103e+00  4.58617216e+00
 -5.36845247e+00  4.32318510e+00  4.09540221e+00  3.93621248e+00
  3.82123584e+00  3.60099326e+00  3.35927710e+00  3.28198129e+00
  3.12675358e+00  3.07467500e+00  3.01416082e+00 -4.99107051e+00
 -4.85630233e+00 -4.58001515e+00 -4.52692452e+00 -4.33751448e+00
 -4.20710906e+00 -4.06885247e+00 -3.98378679e+00 -3.69357057e+00
 -3.60139182e+00 -3.56267845e+00 -3.46010469e+00 -3.31906105e+00
  2.76686531e+00  2.58138639e+00  2.51492666e+00 -3.12241886e+00
 -3.21071378e+00 -3.20636894e+00 -3.00940203e+00 -3.02921781e+00
  2.39821242e+00  2.36382283e+00  2.34152674e+00 -2.71622957e+00
 -2.89886324e+00  2.14781233e+00  2.08903838e+00 -2.64766313e+00
 -2.59052426e+00  1.96336359e+00  1.95158198e+00  1.83340162e+00
 -2.32178115e+00 -2.47320097e+00 -2.44194464e+00  1.71812151e+00
  1.63082698e+00 -2.17020135e+00 -2.12680413e+00 -2.06042540e+00
 -1.99873950e+00